In [10]:
import pandas as pd

df = pd.read_parquet('..\Project Main\data\Appliance_file_subset.parquet')

df.drop(['images', 'asin', 'parent_asin', 'user_id',
       'timestamp', 'helpful_vote', 'verified_purchase'], axis=1, inplace=True)

df.head()



,rating,title,text
0,2.0,"Maybe they be ok for you, but",It was difficult for this old guy to get down ...
1,5.0,Five Stars,Great!
2,5.0,Saves a lot on the cost of coffee.,Works very well. They also clean very easily.
3,5.0,Works great,Easy to use. Easy to remove. Works as it sho...
4,5.0,Humidifier Filters,As described great shape thank you


In [1]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

# Example preprocessing
def encode_review(title, description):
    review_text = title + " " + description
    return tokenizer(review_text, padding=True, truncation=True, return_tensors="tf")

# Example input data
encoded_review = encode_review("Great product", "This product exceeded my expectations.")
output = model(encoded_review['input_ids'])

# Compile the model
model.compile(optimizer=Adam(learning_rate=5e-5), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

# Fit the model on your training data
model.fit(train_dataset, epochs=3)


C:\Users\lonni\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: Failed to import transformers.models.bert.modeling_tf_bert because of the following error (look up to see its traceback):
No module named 'keras.__internal__'

In [4]:
! pip install keras





: 

In [2]:
import numpy as np
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.optimzers import Adam
import pandas as pd


# Load your data
df = pd.read_parquet('..\Project Main\data\Appliance_file_subset.parquet')
df.drop(['images', 'asin', 'parent_asin', 'user_id', 'timestamp', 'helpful_vote', 'verified_purchase'], axis=1, inplace=True)

# Combine the title and text into one feature
df['combined_text'] = df['title'] + ' ' + df['text']

# Prepare tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = TFRobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=5)

# Tokenize the text data
def tokenize_function(texts):
    return tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="tf"
    )

# Prepare the inputs
X = df['combined_text'].tolist()
y = df['rating'].astype(int) - 1  # Ratings should be 0-based for the model (e.g., 0 for 1-star, 4 for 5-star)

# Tokenize the text inputs
X_tokenized = tokenize_function(X)

# Convert tokenized input tensors to numpy arrays
input_ids = X_tokenized['input_ids'].numpy()

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(input_ids, y, test_size=0.2, random_state=42)

# Compile the model
model.compile(optimizer=Adam(learning_rate=5e-5), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

# Train the model
model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=3,
    batch_size=16
)

# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Validation accuracy: {accuracy}')


C:\Users\lonni\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: Failed to import transformers.models.roberta.modeling_tf_roberta because of the following error (look up to see its traceback):
No module named 'keras.__internal__'